# Demostración de la API Python de jpegexp-rs

Este notebook muestra cómo utilizar la API FFI de jpegexp-rs desde Python para codificar y decodificar imágenes JPEG/JPEG2000. Incluye ejemplos de uso, manejo de errores y comparación con bibliotecas estándar.

---

## Requisitos

- Tener compilado el binario `jpegexp_rs.dll` en la carpeta `target/release` o `target/debug`.
- Instalar las siguientes librerías de Python:
    - numpy
    - matplotlib
    - pillow

Puedes instalar los paquetes ejecutando:

```bash
pip install numpy matplotlib pillow
```


In [ ]:
# Instalación de dependencias (ejecuta solo si es necesario)
!pip install numpy matplotlib pillow

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Función para generar un gradiente de prueba

def generate_gradient(width, height):
    arr = np.linspace(0, 255, width, dtype=np.uint8)
    grad = np.tile(arr, (height, 1))
    return grad

In [ ]:
# --- FFI Wrapper para jpegexp-rs ---
import ctypes
from ctypes import POINTER, c_void_p, c_int, c_uint32, c_size_t, c_ubyte

# Buscar DLL
possible_dlls = [
    os.path.join("..", "target", "release", "jpegexp_rs.dll"),
    os.path.join("target", "release", "jpegexp_rs.dll"),
    os.path.join("..", "target", "debug", "jpegexp_rs.dll")
]
DLL_PATH = None
for p in possible_dlls:
    if os.path.exists(p):
        DLL_PATH = os.path.abspath(p)
        break
if DLL_PATH:
    print(f"Usando DLL: {DLL_PATH}")
else:
    print("ADVERTENCIA: No se encontró DLL. Las pruebas FFI fallarán.")

class JpegExpFFI:
    def __init__(self, dll_path):
        if not os.path.exists(dll_path):
            raise RuntimeError(f"DLL no encontrada en {dll_path}")
        self.lib = ctypes.CDLL(dll_path)
        # Tipos
        self.lib.jpegexp_decoder_new.argtypes = [POINTER(c_ubyte), c_size_t]
        self.lib.jpegexp_decoder_new.restype = c_void_p
        self.lib.jpegexp_decoder_free.argtypes = [c_void_p]
        self.lib.jpegexp_decoder_free.restype = None
        class ImageInfo(ctypes.Structure):
            _fields_ = [("width", c_uint32),
                        ("height", c_uint32),
                        ("components", c_uint32),
                        ("bits_per_sample", c_uint32)]
        self.lib.jpegexp_decoder_read_header.argtypes = [c_void_p, POINTER(ImageInfo)]
        self.lib.jpegexp_decoder_read_header.restype = c_int
        self.lib.jpegexp_decoder_decode.argtypes = [c_void_p, POINTER(c_ubyte), c_size_t]
        self.lib.jpegexp_decoder_decode.restype = c_int
        self.lib.jpegexp_encode_jpeg.argtypes = [
            POINTER(c_ubyte), c_uint32, c_uint32, c_uint32,
            POINTER(c_ubyte), c_size_t, POINTER(c_size_t)
        ]
        self.lib.jpegexp_encode_jpeg.restype = c_int
        self.lib.jpegexp_encode_jpegls.argtypes = [
            POINTER(c_ubyte), c_uint32, c_uint32, c_uint32,
            POINTER(c_ubyte), c_size_t, POINTER(c_size_t)
        ]
        self.lib.jpegexp_encode_jpegls.restype = c_int
        self.ImageInfo = ImageInfo
    def decode(self, data):
        data_bytes = (c_ubyte * len(data)).from_buffer_copy(data)
        decoder = self.lib.jpegexp_decoder_new(data_bytes, len(data))
        if not decoder:
            raise RuntimeError("No se pudo crear el decodificador")
        try:
            info = self.ImageInfo()
            res = self.lib.jpegexp_decoder_read_header(decoder, ctypes.byref(info))
            if res != 0:
                raise RuntimeError(f"Fallo al leer cabecera: {res}")
            required_size = info.width * info.height * info.components
            output = (c_ubyte * required_size)()
            res = self.lib.jpegexp_decoder_decode(decoder, output, required_size)
            if res != 0:
                 raise RuntimeError(f"Fallo al decodificar: {res}")
            return bytes(output), info.width, info.height, info.components
        finally:
             self.lib.jpegexp_decoder_free(decoder)
    def encode_jpeg(self, pixels, width, height, components):
         pixel_bytes = (c_ubyte * len(pixels)).from_buffer_copy(pixels)
         out_size = len(pixels) * 2 + 1024
         output = (c_ubyte * out_size)()
         written = c_size_t(0)
         res = self.lib.jpegexp_encode_jpeg(
             pixel_bytes, width, height, components,
             output, out_size, ctypes.byref(written)
         )
         if res != 0:
             raise RuntimeError(f"Fallo al codificar JPEG: {res}")
         return bytes(output[:written.value])

## Ejemplo: Codificación y decodificación de un gradiente

A continuación se muestra cómo codificar una imagen de gradiente en JPEG usando la API FFI y luego decodificarla nuevamente.

In [ ]:
# --- Demostración Directa de la API FFI ---
if DLL_PATH:
    try:
        print("Inicializando FFI...")
        ffi = JpegExpFFI(DLL_PATH)
        # Codificar gradiente
        print("Codificando gradiente en JPEG...")
        w, h = 256, 256
        grad = generate_gradient(w, h)
        grad_bytes = grad.tobytes()
        encoded_jpeg_bytes = ffi.encode_jpeg(grad_bytes, w, h, 1)
        print(f"JPEG codificado: {len(encoded_jpeg_bytes)} bytes.")
        # Verificar con Pillow
        from io import BytesIO
        img = Image.open(BytesIO(encoded_jpeg_bytes))
        print(f"Pillow abrió JPEG: {img.size} {img.mode}")
        plt.imshow(img, cmap='gray')
        plt.title("JPEG codificado por FFI")
        plt.show()
        # Decodificar usando FFI
        print("Decodificando JPEG...")
        decoded_pixels, dw, dh, dc = ffi.decode(encoded_jpeg_bytes)
        print(f"Decodificado: {dw}x{dh}, {dc} componentes.")
        dec_arr = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((dh, dw))
        plt.imshow(dec_arr, cmap='gray')
        plt.title("Pixels decodificados por FFI")
        plt.show()
    except Exception as e:
        print(f"Error en la demo FFI: {e}")
else:
    print("Saltando demo FFI (DLL no encontrada)")

## Manejo de errores y casos especiales

La API FFI puede lanzar excepciones si la DLL no está disponible, si los datos son corruptos o si hay errores internos. Se recomienda envolver las llamadas en bloques `try/except` y validar los datos antes de procesarlos.

## Comparación con bibliotecas estándar

Puedes comparar los resultados de la decodificación con otras bibliotecas como `imagecodecs` o `Pillow` para validar la calidad y compatibilidad de los datos decodificados.

In [ ]:
# Ejemplo de comparación con Pillow
img_pillow = Image.fromarray(generate_gradient(256, 256))
img_pillow.save("grad_pillow.jpg", format="JPEG")
img_loaded = Image.open("grad_pillow.jpg")
plt.imshow(img_loaded, cmap='gray')
plt.title("JPEG generado por Pillow")
plt.show()

## Ejemplo: decodificación de un archivo JPEG2000

A continuación se muestra cómo decodificar un archivo JPEG2000 (.jp2) usando la API FFI y visualizar el resultado. Asegúrate de tener un archivo de prueba en la ruta indicada.

In [ ]:
# Decodificar un archivo JPEG2000 usando la API FFI
jp2_path = "../tests/artifacts_comprehensive/test_images/JPEG2000/CT1.JP2"  # Cambia la ruta a tu archivo
if DLL_PATH and os.path.exists(jp2_path):
    with open(jp2_path, "rb") as f:
        jp2_bytes = f.read()
    try:
        decoded_pixels, w, h, c = ffi.decode(jp2_bytes)
        arr = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w, c) if c > 1 else (h, w))
        plt.imshow(arr, cmap='gray' if c == 1 else None)
        plt.title("JPEG2000 decodificado por FFI")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error al decodificar JPEG2000: {e}")
else:
    print("No se encontró el archivo JPEG2000 o la DLL.")

## Ejemplo: manejo de error con archivo corrupto

Este ejemplo muestra cómo la API FFI responde ante un archivo JPEG2000 corrupto. Se recomienda probar con un archivo que haya causado errores previamente.

In [ ]:
# Intentar decodificar un archivo JPEG2000 corrupto
corrupt_path = "../tests/artifacts_comprehensive/test_images/JPEG2000/bitwiser-icc-corrupted-tagcount-1999.jp2"  # Cambia la ruta si es necesario
if DLL_PATH and os.path.exists(corrupt_path):
    with open(corrupt_path, "rb") as f:
        corrupt_bytes = f.read()
    try:
        decoded_pixels, w, h, c = ffi.decode(corrupt_bytes)
        arr = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w, c) if c > 1 else (h, w))
        plt.imshow(arr, cmap='gray' if c == 1 else None)
        plt.title("JPEG2000 corrupto decodificado (¡debería fallar!)")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error esperado al decodificar archivo corrupto: {e}")
else:
    print("No se encontró el archivo corrupto o la DLL.")

## Ejemplo: codificación JPEG con imagen RGB

Este ejemplo muestra cómo codificar una imagen RGB generada en numpy usando la API FFI y visualizar el resultado con Pillow.

In [ ]:
# Codificar una imagen RGB con la API FFI
if DLL_PATH:
    try:
        # Crear imagen RGB de prueba
        w, h = 128, 128
        rgb = np.zeros((h, w, 3), dtype=np.uint8)
        rgb[..., 0] = np.linspace(0, 255, w, dtype=np.uint8)  # R
        rgb[..., 1] = np.linspace(255, 0, w, dtype=np.uint8)  # G
        rgb[..., 2] = 128  # B
        rgb_bytes = rgb.tobytes()
        encoded_rgb_jpeg = ffi.encode_jpeg(rgb_bytes, w, h, 3)
        print(f"JPEG RGB codificado: {len(encoded_rgb_jpeg)} bytes.")
        from io import BytesIO
        img_rgb = Image.open(BytesIO(encoded_rgb_jpeg))
        plt.imshow(img_rgb)
        plt.title("JPEG RGB codificado por FFI")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error al codificar imagen RGB: {e}")
else:
    print("Saltando ejemplo RGB (DLL no encontrada)")

## Comparación visual: jpegexp-rs vs Pillow

Este ejemplo compara visualmente la decodificación de un JPEG generado por Pillow y decodificado tanto con Pillow como con jpegexp-rs (FFI).

In [ ]:
# Comparación visual entre Pillow y jpegexp-rs (FFI)
if DLL_PATH:
    try:
        # Generar y guardar JPEG con Pillow
        grad = generate_gradient(256, 256)
        img_pillow = Image.fromarray(grad)
        img_pillow.save("grad_pillow_cmp.jpg", format="JPEG")
        # Decodificar con Pillow
        img_loaded = Image.open("grad_pillow_cmp.jpg")
        # Decodificar con FFI
        with open("grad_pillow_cmp.jpg", "rb") as f:
            jpeg_bytes = f.read()
        decoded_pixels, w, h, c = ffi.decode(jpeg_bytes)
        arr_ffi = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w))
        # Mostrar lado a lado
        fig, axs = plt.subplots(1, 2, figsize=(10, 4))
        axs[0].imshow(img_loaded, cmap='gray')
        axs[0].set_title("Decodificado con Pillow")
        axs[0].axis('off')
        axs[1].imshow(arr_ffi, cmap='gray')
        axs[1].set_title("Decodificado con jpegexp-rs (FFI)")
        axs[1].axis('off')
        plt.show()
    except Exception as e:
        print(f"Error en la comparación visual: {e}")
else:
    print("Saltando comparación visual (DLL no encontrada)")

## Comparación cuantitativa: PSNR entre Pillow y jpegexp-rs

Calcula el PSNR (Peak Signal-to-Noise Ratio) entre la imagen decodificada por Pillow y la decodificada por jpegexp-rs para cuantificar la diferencia.

In [ ]:
# Calcular PSNR entre Pillow y jpegexp-rs (FFI)
def psnr(img1, img2):
    mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

if DLL_PATH:
    try:
        # Usar los mismos archivos del ejemplo anterior
        grad = generate_gradient(256, 256)
        img_pillow = Image.fromarray(grad)
        img_pillow.save("grad_pillow_cmp.jpg", format="JPEG")
        img_loaded = Image.open("grad_pillow_cmp.jpg")
        arr_pillow = np.array(img_loaded)
        with open("grad_pillow_cmp.jpg", "rb") as f:
            jpeg_bytes = f.read()
        decoded_pixels, w, h, c = ffi.decode(jpeg_bytes)
        arr_ffi = np.frombuffer(decoded_pixels, dtype=np.uint8).reshape((h, w))
        score = psnr(arr_pillow, arr_ffi)
        print(f"PSNR entre Pillow y jpegexp-rs: {score:.2f} dB")
    except Exception as e:
        print(f"Error al calcular PSNR: {e}")
else:
    print("Saltando cálculo de PSNR (DLL no encontrada)")